In [1]:
import pandas as pd


In [25]:
df = pd.read_excel('./data/westmetall arch/westmetall_raw.xlsx', sheet_name='AL')
df.shape

(4170, 4)

In [26]:
columns = df.columns.to_list()
columns

['date',
 'LME Aluminium Cash-Settlement',
 'LME Aluminium 3-month',
 'LME Aluminium stock']

In [27]:
date = columns[0]
settlment = columns[1]
month = columns[2]
stock = columns[3]

In [28]:
df_cleared = df[~(df['date'] == 'date')]
df_cleared = df_cleared[~(df_cleared['date'] == 'Aluminium')]
df_cleared.dropna(inplace=True)
df_cleared.reset_index(drop=True, inplace=True)

In [29]:
df_cleared

,date,LME Aluminium Cash-Settlement,LME Aluminium 3-month,LME Aluminium stock
0,31. July 2023,"2,195.50","2,242.00",507.4
1,28. July 2023,"2,161.50","2,203.00",509
2,27. July 2023,"2,176.00","2,220.00",511
3,26. July 2023,"2,170.50","2,215.00",514.025
4,25. July 2023,"2,184.00","2,226.50",516.975
...,...,...,...,...
3934,08. January 2008,"2,442.00","2,491.50",935.15
3935,07. January 2008,"2,404.00","2,459.00",934.35
3936,04. January 2008,"2,447.00","2,498.00",933.75
3937,03. January 2008,"2,422.00","2,470.50",929.5


In [30]:
df_cleared['date'] = pd.to_datetime(df_cleared['date'])


In [31]:
df_cleared[settlment] = df_cleared[settlment].str.replace(',','')
df_cleared[settlment] = df_cleared[settlment].str.replace('-','')

df_cleared[month] = df_cleared[month].str.replace(',','')
df_cleared[month] = df_cleared[month].str.replace('-','')

df_cleared[stock] = df_cleared[stock].str.replace(',','.')
df_cleared[stock] = df_cleared[stock].str.replace('-','')

In [32]:
df_cleared[settlment] = pd.to_numeric(df_cleared[settlment])
df_cleared[month] = pd.to_numeric(df_cleared[month])
df_cleared[stock] = pd.to_numeric(df_cleared[stock])

ValueError: Unable to parse string "1.000.375" at position 433

In [33]:
df_cleared.iloc[433]

date                             2021-11-08 00:00:00
LME Aluminium Cash-Settlement                 2530.5
LME Aluminium 3-month                         2558.0
LME Aluminium stock                        1.000.375
Name: 433, dtype: object

In [46]:
df_cleared

,date,LME Zinc Cash-Settlement,LME Zinc 3-month,LME Zinc stock
0,2023-07-31,2550.00,2540.00,NaN
1,2023-07-28,2485.00,2482.00,NaN
2,2023-07-27,2485.00,2487.00,NaN
3,2023-07-26,2455.00,2466.00,NaN
4,2023-07-25,2447.00,2456.00,NaN
...,...,...,...,...
3934,2008-01-08,2534.00,2540.50,NaN
3935,2008-01-07,2475.50,2482.00,NaN
3936,2008-01-04,2563.00,2555.00,NaN
3937,2008-01-03,2466.00,2477.00,NaN


In [47]:
with pd.ExcelWriter(
    "./data/westmetall_zink.xlsx",
        date_format="YYYY-MM-DD",
        datetime_format="YYYY-MM-DD") as writer:
    df_cleared.to_excel(writer, sheet_name='Zink_cleared')
